In [ ]:
import tensorflow as tf
import keras
from keras.applications.resnet import ResNet50, preprocess_input
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.densenet import DenseNet121, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

In [ ]:
!pip3 install Py-FS

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 15.6 MB/s 


In [ ]:
!pip install Relieff

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48 kB 4.1 MB/s 
  Created wheel for Relieff: filename=ReliefF-0.1.2-py3-none-any.whl size=6319 sha256=00f52294fc533193af87eca7fef0674adc44ebf8c43604eda49ef486b70f2b76
  Stored in directory: /root/.cache/pip/wheels/18/8a/4a/3aada56cb869df4c1b1ed6af509c941a25985d1d1258a2e78c
Successfully built Relieff


In [ ]:
from Py_FS.filter import MI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_root = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/lung_image_sets', labels='inferred')
datasets = input_root.class_names
datasets
print(input_root)

Found 15000 files belonging to 3 classes.
<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
test_set = [0.25, 0.25, 0.25]

In [ ]:
def check_if_directory_exists(name_folder):
    if not os.path.exists(name_folder):
        print(name_folder + " directory does not exist, created")
        os.makedirs(name_folder) 
    else:
        print(name_folder + " directory exists, no action performed")

In [ ]:
def load_json_file(name_json):
    """
    load_json_file(name_json)
    INPUT:
        name_json: name of the json file to be loaded
    OUTPUT:
        variable with the json file loaded
        
    @author: Eduardo Fidalgo (EFF)
    """
    import json
    with open(name_json) as f:    
      config = json.load(f)
      
    return config

In [ ]:
def os_path_separator():
    """
    load_json_file(name_json)
    INPUT:
        None
    OUTPUT:
        path_sep: variable containing the windows or linux path separator
        
    @author: Eduardo Fidalgo (EFF)
    """
    from sys import platform as _platform
    
    # print start time
    print ("[INFO] Platform where script is being used- " + str(_platform))
    
    # Patch separator
    if _platform  == "win64" or _platform  == "win32":
        path_sep = "\\"
    else:
        path_sep = "/"
    
    return path_sep

def read_image_names(path_ima='./images',img_ext='*.png',verbose=False):
    """
    read_image_names(path_ima='./images', img_ext='*.png',verbose=False)
    Given a path and an image file extension, this function introduce all
    the image name contained in the folder passed in the path in a list and
    it returns it. The introduced name will contain the relative path
    (the directory concatenate with the image name).
    INPUT:
        path_imag: string containing the absolute or relative path.
            By default path_imag= './images'
        img_ext: string containing the extension of the image files to be read.
            By default img_ext='*.png'
        verbose: boolean. If True, messages are shown.
            By default verbose=False
    OUTPUT:
        full_names_img: list containing file names with its path (in a sigle
        string).
    Extracted from ktools
    """
    import os
    import glob

    # List to be returned
    full_names_img = list()

    #% % 1_ Print the original path and check if the folder with images exists!
    prev_path = os.getcwd()
    if verbose:
        print('(1) Original path was:  {} \n'.format(prev_path))

    if os.path.isdir(path_ima):
        if verbose:
            print('(2) The directory {} does exists!\n'.format(path_ima))
    else:
        if verbose:
            print('(2) WARNING: There is not such directory\n')


    #% % 2_ Read all the files with the specified "extension" in the current
    # working directory
    full_names_img = glob.glob(os.path.join(path_ima,img_ext))
    # glob.glob(os.path.join(path_img, img_ext))

    if verbose:
        print('(3) The "{0}" directory has {1} images with {2} extension.\n'
              .format(os.getcwd(),len(full_names_img),img_ext))

    return full_names_img



In [ ]:
from keras.preprocessing import image

from keras.utils import load_img, img_to_array

In [ ]:
from zmq.sugar.socket import T


for ds in range(0, len(datasets)):
    
    # Select the configuration file available
    #file = 'conf_' + dataset[ds] + '.json'
    
    # load the user configs. It has been created a json file per dataset
    #with open(file) as f:    
    #  config = json.load(f)
      
    # config variables
    model_name    = "desnet121"
    weights       = "imagenet"
    include_top   = 0 
    train_path    = "/content/drive/MyDrive/lung_image_sets/"
    features_path = "output/" + datasets[ds] + "/" + model_name + "/features.h5" 
    labels_path   = "output/" + datasets[ds] + "/" + model_name + "/labels.h5"
    results       = "output/" + datasets[ds] + "/" + model_name + "/results" + datasets[ds] + ".txt"
    model_path    = "output/" + datasets[ds] + "/" + model_name + "/model"
    test_size     = test_set[ds]
    
    print("t ",train_path)
    print("p")
    print(len(train_path))
    print(train_path)


    # check if the output directory exists, if not, create it.
    check_if_directory_exists("output")
    
    # check if the output directory exists, if not, create it.
    check_if_directory_exists("output/" + datasets[ds])
    
    # start time
    print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
    start = time.time()
    
    # create the pretrained models
    # check for pretrained weight usage or not
    # check for top layers to be included or not
    if model_name == "vgg16":
      base_model = VGG16(weights=weights)
      model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc1').output)
      image_size = (224, 224)
    elif model_name == "desnet121":
      base_model = DenseNet121(weights=weights)
      model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)
      image_size = (224, 224)
    elif model_name == "resnet50":
      base_model = ResNet50(weights=weights)
      model = Model(inputs=base_model.input, outputs=base_model.get_layer('conv_5b').output)
      image_size = (224, 224)
    elif model_name == "inceptionv3":
      base_model = InceptionV3(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
      model = Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)
      image_size = (224, 224)
    elif model_name == "mobilenet":
      base_model = MobileNet(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3))
      model = Model(inputs=base_model.input, outputs=base_model.get_layer('conv1_relu').output)
      image_size = (224, 224)
    else:
      base_model = None
    
    # check if the output directory exists, if not, create it.
    check_if_directory_exists("output/" + datasets[ds] + "/" + model_name)
    
    print ("[INFO] successfully loaded base model and model...")
    check_if_directory_exists("datasets/" + datasets[ds])

    # path to training dataset
    train_labels = os.listdir(train_path)
    print("tl ",train_labels)

    # encode the labels
    print ("[INFO] encoding labels...")
    le = LabelEncoder()
    le.fit([tl for tl in train_labels])
    
    # variables to hold features and labels
    features = []
    labels   = []
#     print(train_labels)
#     # loop over all the labels in the folder
    count = 1
    for i, label in enumerate(train_labels):   
      cur_path = train_path + "/" + label
       # check how many files are, together with their extensions
      list_files = os.listdir(cur_path)  
      count = 1
      #for image_path in glob.glob(cur_path + "/*.jpg"):
      for image_path in range(0, len(list_files)):
        #print ("[INFO] Processing - " + str(count) + " named " + list_files[image_path])
        img = load_img(cur_path + "/" + list_files[image_path], target_size=image_size)
        x = img_to_array(img) 
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        print(x)
        features.append(flat)
        print(features)
        labels.append(label)
        print ("[INFO] processed for image - " + list_files[image_path])
        count += 1
      print ("[INFO] completed label - " + label)
    
    # encode the labels using LabelEncoder
    le = LabelEncoder()
    le_labels = le.fit_transform(labels)
    
    # get the shape of training labels
    print ("[STATUS] training labels: {}".format(le_labels))

    print ("[STATUS] training labels shape: {}".format(le_labels.shape))
    
    # save features and labels
    try:
        h5f_data = h5py.File(features_path, 'w')
    except:
        a=1;
        
    h5f_data.create_dataset('dataset_1', data=np.array(features))
    
    h5f_label = h5py.File(labels_path, 'w')
    h5f_label.create_dataset('dataset_1', data=np.array(le_labels))
    
    h5f_data.close()
    h5f_label.close()
    
    # save model and weights
    model_json = model.to_json()
    with open(model_path + str(test_size) + ".json", "w") as json_file:
      json_file.write(model_json)
    
    # save weights
    model.save_weights(model_path + str(test_size) + ".h5")
    print("[STATUS] saved model and weights to disk..")
    
    print ("[STATUS] features and labels saved..")
    print(features)
    
    # end time
    end = time.time()
    print ("[STATUS] end time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

Streaming output truncated to the last 5000 lines.
       4.4409105e-01, 1.5926011e-02], dtype=float32), array([3.8693383e-04, 1.1176204e-02, 1.2930454e-03, ..., 8.6521018e-01,
       3.3615828e-01, 1.0861247e-01], dtype=float32), array([6.9252524e-04, 9.6335122e-03, 8.1538965e-05, ..., 1.5382470e+00,
       2.6622501e-01, 2.6425030e-03], dtype=float32), array([6.8729324e-04, 1.1372425e-02, 8.9231011e-04, ..., 1.6974792e+00,
       4.4424245e-01, 3.1575270e-02], dtype=float32), array([6.0493109e-04, 7.5218566e-03, 1.3077694e-04, ..., 1.2221711e+00,
       5.8964801e-01, 1.2556283e-02], dtype=float32), array([5.4900476e-04, 6.6951974e-03, 6.2138983e-04, ..., 1.6091675e+00,
       6.3331878e-01, 2.8488327e-02], dtype=float32), array([4.2577830e-04, 1.0187440e-02, 2.3015779e-03, ..., 1.4075679e+00,
       1.7975135e-01, 0.0000000e+00], dtype=float32), array([4.6865857e-04, 1.2175068e-02, 8.6194318e-04, ..., 6.0669231e-01,
       2.9807463e-01, 0.0000000e+00], dtype=float32), array([3.5081

In [ ]:
features

array([], dtype=float64)

In [ ]:
from __future__ import print_function

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics
from sklearn import svm
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import statistics as st
import matplotlib.pyplot as plt


In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 15 21:40:17 2018
@author: Eduardo Fidalgo
"""

# organize imports
from __future__ import print_function

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import svm
import numpy as np
import h5py
import os
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt


config = model.to_json()
with open('conf_' + datasets[ds] + '.json', 'w') as json_file:
  json_file.write(config)

# test_size     = config["test_size"]
# seed      = config["seed"]
# features_path   = config["features_path"]
# labels_path   = config["labels_path"]
# results     = config["results"]
# classifier_path = config["classifier_path"]
# train_path    = config["train_path"]
# num_classes   = config["num_classes"]

# import features and labels

h5f_data  = h5py.File(features_path, 'r')
h5f_label = h5py.File(labels_path, 'r')

features_string = h5f_data['dataset_1']
labels_string   = h5f_label['dataset_1']
print(h5f_data)
features = np.array(features_string)
labels   = np.array(labels_string)

h5f_data.close()
h5f_label.close()

# verify the shape of features and labels
print ("[INFO] features shape: {}".format(features.shape))
print ("[INFO] labels shape: {}".format(labels.shape))

print ("[INFO] training started...")
# split the training and testing data
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(features),
                                                                  np.array(labels),
                                                                  test_size=test_size,
                                                                  random_state=None)

print ("[INFO] splitted train and test data...")
print ("[INFO] train data  : {}".format(trainData.shape))
print ("[INFO] test data   : {}".format(testData.shape))
print ("[INFO] train labels: {}".format(trainLabels.shape))
print ("[INFO] test labels : {}".format(testLabels.shape))

# use logistic regression as the model
print ("[INFO] creating model...")
#model = LogisticRegression(random_state=seed)
model = svm.SVC(kernel='linear', probability=True, class_weight='balanced')
model.fit(trainData, trainLabels)

# use rank-1 and rank-5 predictions
print ("[INFO] evaluating model...")
f = open(results, "w")
rank_1 = 0
rank_5 = 0

# loop over test data
for (label, features) in zip(testLabels, testData):
  # predict the probability of each class label and
  # take the top-5 class labels
  predictions = model.predict_proba(np.atleast_2d(features))[0]
  predictions = np.argsort(predictions)[::-1][:5]

  # rank-1 prediction increment
  if label == predictions[0]:
    rank_1 += 1

  # rank-5 prediction increment
  if label in predictions:
    rank_5 += 1

# convert accuracies to percentages
rank_1 = (rank_1 / float(len(testLabels))) * 100
rank_5 = (rank_5 / float(len(testLabels))) * 100

# write the accuracies to file
f.write("Rank-1: {:.2f}%\n".format(rank_1))
f.write("Rank-5: {:.2f}%\n\n".format(rank_5))

# evaluate the model of test data
preds = model.predict(testData)

# write the classification report to file
f.write("{}\n".format(classification_report(testLabels, preds)))
f.close()

# dump classifier to file
print ("[INFO] saving model...")
pickle.dump(model, open(classifier_path, 'wb'))

# display the confusion matrix
print ("[INFO] confusion matrix")

# get the list of training lables
labels = sorted(list(os.listdir(train_path)))

# plot the confusion matrix
cm = confusion_matrix(testLabels, preds)
sns.heatmap(cm,
            annot=True,
            cmap="Set2")
plt.show()

<HDF5 file "features.h5" (mode r)>
[INFO] features shape: (0,)
[INFO] labels shape: (0,)
[INFO] training started...


ValueError: ignored

In [ ]:
def __init__(self, classifiers=None, meta_classifier=None, 
            use_probability=False, double_down=False, 
            average_probs=False, clones=True, verbose=2):
  self.classifiers = classifiers
  self.meta_classifier = meta_classifier
  self.use_probability = use_probability
  self.double_down = double_down
  self.average_probs = average_probs
  self.clones = clones
  self.verbose = verbose

In [ ]:
def fit(self, features, labels, sample_weight=None):
  start = time.time()
  if self.clones:
    self.clfs_ = clone(self.classifiers)
    self.meta_clf_  = clone(self.meta_classifier)
  else:
    self.clfs_ = self.classifiers
    self.meta_clf_ = self.meta_classifier
    
  if self.verbose > 0:
    print('Fitting %d classifiers' % (len(self.classifiers)))
  n=1
  for clf in self.clfs_:
    if self.verbose > 1:
      print(f"Fitting classifier {n}/{len(self.clfs_)}")
      n +=1
    if sample_weight is None:
      clf.fit(features ,labels)
    else:
      clf.fit(features, labels, sample_weight)
  meta_features = self.predict_meta(features)
  if verbose >1:
    print("Fitting meta-classifier to meta_features")
  elif sparse.issparse(features):
    meta_features = sparse.hstack((features, meta_features))
  else:
    meta_features = np.hstack((features, meta_features))
  self.meta_features_ = meta_features

  if sample_weight is None:
    self.meta_clf_.fit(meta_features, labels)
  else:
    self.meta_clf_.fit(meta_features, labels, sample_weight=sample_weight)

  stop = time.time()
  if verbose > 0:
    print(f"Estimators Fit! Time Elapsed: {(stop-start)/60} minutes")

  return self



In [ ]:
def predict_meta(self, features):
  if self.use_probability:
    probs = np.asarray([clf.predict_probs(features) for clf in self.clfs_])
    if self.average_probs:
      preds = np.average(probs, axis=0)

    else:
      preds = np.concatenate(probs, axis=1)

  else:
    preds = np.column_stack([clf.predict(features) for clf in self.clfs_])
        
  return preds

def predict_probs(self, features):
  meta_features = self.predict_meta(features)

  if self.double_down == False:
    return self.meta_clf_.predict_probs(meta_features)
        
  elif sparse.issparse(features):
    return self.meta_clf_.predict_probs(sparse.hstack((features, meta_features)))

  else:
    return self.meta_clf_.predict_probs(np.hstack((features, meta_features)))


def predict(self, features):
  meta_features = self.predict_meta(features)
  if self.double_down == False:
    return self.meta_clf_.predict(meta_features)

  elif sparse.issparse(features):
    return self.meta_clf_.predict(sparse.hstack((features, meta_features)))

  else:
    return self.meta_clf_.predict(np.hstack((features, meta_features)))